In [40]:
import math
import csv
from scipy.spatial.transform import Rotation as R
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import rosbag
from sensor_msgs.msg import Imu, NavSatFix
import scipy.io as sio
import numpy as np
from collections import OrderedDict

In [41]:
data_path = '/home/crange/dev/calib_ws/datas/calib_data_52/degree2radian/XYZ'

In [42]:
original_file = data_path + '/odometry.txt'

f = open(original_file, encoding="utf-8")

data_line = f.readline()
original_datas = []
while data_line:
    d = data_line.replace('\n', '').split()
    original_datas.append(d)
    data_line = f.readline()

f.close()

 - odometry_loc.txt modify


In [43]:
modify = 0
sub = -1.0
output_data = []
base_position = [float(original_datas[0][1]), float(original_datas[0][2]), float(original_datas[0][3])]
for i in range(len(original_datas)):
    timestamp = float(original_datas[i][0])
    # x = float(original_datas[i][1]) - base_position[0]
    # y = float(original_datas[i][2]) - base_position[1]
    # z = float(original_datas[i][3]) - base_position[2]
    x = float(original_datas[i][1])
    y = float(original_datas[i][2])
    z = float(original_datas[i][3])
    qx = float(original_datas[i][4])
    qy = float(original_datas[i][5])
    qz = float(original_datas[i][6])
    qw = float(original_datas[i][7])

    r = R.from_quat([qx, qy, qz, qw])
    euler_angle = r.as_euler('xyz', degrees=True)
    euler_angle[2] = modify + sub * euler_angle[2]

    r = R.from_euler('xyz', euler_angle, degrees=True)
    q = r.as_quat()

    output_data.append([timestamp, x, y, z, q[0], q[1], q[2], q[3]])

In [44]:
save_file = data_path + '/odometry_.txt'
fout = open(save_file, 'w', encoding='utf-8')
for i in range(len(output_data)):
    print('{:.{}f}\t{:.{}f}\t{:.{}f}\t{:.{}f}\t{:.{}f}\t{:.{}f}\t{:.{}f}\t{:.{}f}'
          .format(output_data[i][0], 9,
                  output_data[i][1], 9,
                  output_data[i][2], 9,
                  output_data[i][3], 9,
                  output_data[i][4], 9,
                  output_data[i][5], 9,
                  output_data[i][6], 9,
                  output_data[i][7], 9), file=fout)

fout.close()